<a href="https://colab.research.google.com/github/yashgsa03/Download/blob/main/RandomForestEEGState.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
!pip install liac-arff
!pip install mne # Install the missing mne module

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=3930b4e19f01784b50648c41ac4efc74980512983b6ed6a853237ea9cfd5b9f8
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.4 MB/s eta 0:00:00


# Convert ARFF to CSV

In [ ]:
import arff
import csv

with open('./EEG Eye State.arff', 'r') as f:
    arff_data = arff.load(f)

with open('output.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([attribute[0] for attribute in arff_data['attributes']]) # Access the attributes list using a key
    for row in arff_data['data']: # Access the data list using a key
        writer.writerow(row)

In [ ]:
import pandas as pd
from scipy.io import arff
import numpy as np
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from mne.preprocessing import ICA # Now you can import ICA from mne
import mne

# Step 1: Convert .arff to .csv
def convert_arff_to_csv(arff_file_path, csv_file_path):
    # Try opening the file directly to check if it exists and is accessible
    try:
        with open(arff_file_path, 'r') as f:
            data, meta = arff.loadarff(f)
            df = pd.DataFrame(data)
            df.to_csv(csv_file_path, index=False)
        print("Conversion successful!")
    except FileNotFoundError:
        print(f"Error: File not found at {arff_file_path}")
    except PermissionError:
        print(f"Error: Permission denied to access {arff_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# File paths (Update these if necessary)
arff_file_path = './EEG Eye State.arff'  # Replace with the correct path
csv_file_path = 'output.csv'  # Replace with the desired output path

# Convert ARFF to CSV
convert_arff_to_csv(arff_file_path, csv_file_path)

Conversion successful!


# Load and Pre-Process CSV File Data

In [ ]:
# Step 2: Load and preprocess data
def load_eeg_data(file_path):
    try:
        data = pd.read_csv(file_path)
        print("CSV file loaded successfully!")
        return data
    except FileNotFoundError:
        print(f"Error: CSV file not found at {file_path}. Make sure the file exists and the path is correct.")
        return None

def bandpass_filter(data, lowcut=0.1, highcut=30, fs=128, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

# Load CSV data
raw_data = load_eeg_data(csv_file_path)


if raw_data is not None:
    # Apply bandpass filter (assuming bandpass_filter is defined elsewhere)
    print('......filtering data................')
    filtered_data = raw_data.iloc[:, :-1].apply(lambda x: bandpass_filter(x,0.1,30), axis=0)  # Exclude the label column

    # ... (Rest of your code)
else:
    print("Error loading data. Further processing is skipped.")

def remove_artifacts_ica(data, sfreq=128, n_components=None): # Set n_components to None
    ch_names = [f'EEG {i+1}' for i in range(data.shape[1])]
    raw = mne.io.RawArray(data.T, mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg'))
    # Determine number of components automatically if not specified
    if n_components is None:
        n_components = min(data.shape[1], 14) # Set to the number of channels or 14, whichever is smaller
    ica = ICA(n_components=n_components, random_state=97, max_iter=800).fit(raw)
    return ica.apply(raw).get_data().T

# Apply bandpass filter
#filtered_data = raw_data.iloc[:, :-1].apply(lambda x: bandpass_filter(x), axis=0)  # Exclude the label column

# Remove artifacts using ICA
#clean_data = remove_artifacts_ica(filtered_data.values,128,14) # Now this should work without error

CSV file loaded successfully!
......filtering data................


# Train and Predict

In [ ]:

# Prepare data for training
labels = raw_data.iloc[:, -1]  # Assuming the last column is the label
features = pd.DataFrame(filtered_data, columns=raw_data.columns[:-1])

# Split and normalize data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train, X_test = sxcaler.transform(X_train), scaler.transform(X_test)

# Train
# Train and evaluate model
clf = RandomForestClassifier(n_estimators=150, random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Print results
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9392523364485982
Classification Report:
              precision    recall  f1-score   support

        b'0'       0.94      0.94      0.94      1586
        b'1'       0.94      0.94      0.94      1410

    accuracy                           0.94      2996
   macro avg       0.94      0.94      0.94      2996
weighted avg       0.94      0.94      0.94      2996

